##Welcome to my NYC Airbnb EDA project - my second EDA project.##

We will validate the data to make sure the .csv file is appropriate to work with, and if not then we will clean the data and handle the issues that we might run into with Pandas.

After the validation we analyze the data for any underlying trends and interesting metrics that we want to investigate, and I will present these findings visually through: Matplotlib, Seaborn, and Tableau

In [1]:
import pandas as pandas

# Here is where we will investigate and validate the data for completeness and correctness.



